In [1]:
# Cargo los datos sobre los que quiero hacer predicciones
import cx_Oracle
import pandas as pd


In [19]:

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select periodo,product_id,sum(tn) tn from PC.L_VM_COMPLETA join L_PRODUCTOS_A_PREDECIR using(product_id) group by periodo,product_id order by product_id,periodo" 
df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_pred_full = pd.concat(df_pred, ignore_index=True)
conn.close()

C:\Users\pablo\AppData\Local\Temp\ipykernel_9736\1234798557.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM


In [20]:
print(df_pred_full.shape)

(16088, 3)


In [22]:
# convertir el periodo a datetime
df_pred_full['PERIODO'] = pd.to_datetime(df_pred_full['PERIODO'], format='%Y%m')

# Para cada producto si faltan meses entre 201701 y 201912, reemplazar con 0 el tn de esos meses para cada producto
def complete_months(df, start_date, end_date):
    # Crear un rango de fechas mensuales
    all_months = pd.date_range(start=start_date, end=end_date, freq='MS')
    
    # Crear un DataFrame con todos los meses y los productos únicos
    products = df['PRODUCT_ID'].unique()
    all_combinations = pd.MultiIndex.from_product([products, all_months], names=['PRODUCT_ID', 'PERIODO'])
    complete_df = pd.DataFrame(index=all_combinations).reset_index()
    
    # Unir con el DataFrame original
    complete_df = complete_df.merge(df, on=['PRODUCT_ID', 'PERIODO'], how='left')
    
    # Rellenar los valores NaN con 0
    complete_df['TN'] = complete_df['TN'].fillna(0)
    
    return complete_df
# Completar los meses entre 201701 y 201912
df_complete = complete_months(df_pred_full, '2017-01-01', '2019-12-01')
print(df_complete.shape)



(28080, 3)


In [29]:
# ...existing code de carga y preprocesamiento...

from statsforecast import StatsForecast
from statsforecast.models import SARIMA
import polars as pl


C:\Users\pablo\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'SARIMA' from 'statsforecast.models' (c:\anaconda3\envs\Autogluon\lib\site-packages\statsforecast\models.py)

In [ ]:

# Convertir el DataFrame a formato polars y renombrar columnas para statsforecast
df_sf = pl.from_pandas(df_complete.rename(columns={
    'PRODUCT_ID': 'unique_id',
    'PERIODO': 'ds',
    'TN': 'y'
}))

# Crear el objeto StatsForecast
sf = StatsForecast(
    models=[SARIMA(order=(1,1,0), seasonal_order=(1,1,0,12))],
    freq='M',  # Mensual
    n_jobs=-1
)

# Ajustar y predecir los próximos 2 meses para cada serie
# last_window_size es opcional, puedes omitirlo si tienes todos los datos completos
df_forecast = sf.forecast(df=df_sf, h=2)

# Tomar solo el segundo mes pronosticado para cada producto
df_result = df_forecast.filter(pl.col('ds') == df_sf['ds'].max() + pl.duration(months=2))

# Renombrar columnas para coincidir con tu formato original
df_result = df_result.rename({'unique_id': 'PRODUCT_ID', 'SARIMA': 'TN_PREDICTED'})

# Convertir a pandas y guardar
df_result = df_result.to_pandas()
df_result['TN_PREDICTED'] = df_result['TN_PREDICTED'].clip(lower=0)
df_result.to_csv('pronostico_statsforecast.csv', index=False, encoding='utf-8-sig')

print(df_result.head())

In [27]:
# Hacer cero los valores de tn_predicted que sean menores a cero
df_pronostico['TN_PREDICTED'] = df_pronostico['TN_PREDICTED'].clip(lower=0) 
# Guardar el DataFrame de pronósticos en un archivo CSV
df_pronostico.to_csv('pronostico_sarimax_2.csv', index=False, encoding='utf-8-sig')   

In [28]:
print(df_pronostico.head())  # Mostrar las primeras filas del DataFrame de pronósticos


  PRODUCT_ID  TN_PREDICTED
0      20001   2460.547605
1      20002   1512.242450
2      20003   1585.858222
3      20004    950.804368
4      20005    844.389682
